## OBJECTIVES
1. Clean the data:
- Handle missing values ​​(null).
- Standardize date formats.
- Correct spelling errors in product or category names.
- Convert data types appropriate for analysis.

2. Analyze the data:
- Calculate total revenue by day.
- Identify best-selling products.
- Analyze revenue by time frame of day.
- Calculate average number of products sold per transaction.

3. Visualize the data:
- Plot revenue by day.
- Column chart for top 5 best-selling products.
- Pie chart showing revenue percentage by product category.

In [1]:
# Import needed libraries

import kagglehub 
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np

In [2]:
# Download latest version data from kagglehub
path = kagglehub.dataset_download("ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training")
print("Path to dataset files:", path)

100%|█████████████████████████████████████████████████████████████████████████████████| 111k/111k [00:00<00:00, 6.34MB/s]

Extracting files...
Path to dataset files: /home/gitpod/.cache/kagglehub/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training/versions/1


In [3]:
# Show list of files in kaggle hub folder

file_path = '/home/gitpod/.cache/kagglehub/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training/versions/1'
all_files = os.listdir(file_path)
print('This folder includes the following files:')
for file in all_files:
    print(file)

This folder includes the following files:
dirty_cafe_sales.csv


In [5]:
# Read csv file

raw = pd.read_csv('/home/gitpod/.cache/kagglehub/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training/versions/1/dirty_cafe_sales.csv')

In [7]:
# Have a look throught the dataframe

raw.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [8]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [13]:
# Change data type

raw['Quantity'] = pd.to_numeric(raw['Quantity'], errors='coerce')
raw['Price Per Unit'] = pd.to_numeric(raw['Price Per Unit'], errors='coerce')
raw['Transaction Date'] = pd.to_datetime(raw['Transaction Date'], errors='coerce')

# error = coerce to change all Error data to NaN

In [15]:
# Check if there are NaN value in the dataframe

raw.isna().sum()

Transaction ID         0
Item                 333
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      2579
Location            3265
Transaction Date     460
dtype: int64

In [27]:
# Drop NaN in Item 

raw = raw.dropna(subset = ['Item'])

In [31]:
# Drop UNKOWN in Item 

raw = raw[raw['Item'] != "UNKNOWN"]

In [32]:
# After verifying each item (using raw[raw['Item'] == 'Cake'], etc ... ), 
# it is confirmed that each item has a unique value for 'Price per unit'.

# There are 533 NaN in the 'Price per unit' column, 
# which can be filled using the corresponding item's known price.

# Create a dictionary mapping each Item to its respective Price per unit.

In [33]:
raw_copy = raw.copy()

In [34]:
item_to_price = {}
for item, group in raw_copy.groupby('Item'):
    price = group['Price Per Unit'].dropna()
    item_to_price[item] = price.iloc[0]

# When using for loop with .groupby, python return each loop with (key, group)
# 

In [35]:
item_to_price

{'Cake': np.float64(3.0),
 'Coffee': np.float64(2.0),
 'Cookie': np.float64(1.0),
 'ERROR': np.float64(1.5),
 'Juice': np.float64(3.0),
 'Salad': np.float64(5.0),
 'Sandwich': np.float64(4.0),
 'Smoothie': np.float64(4.0),
 'Tea': np.float64(1.5)}